### Experiment 2 - Random Forest

In [8]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score, f1_score, precision_score, recall_score

import mlflow
import dagshub
import logging

print('Setup Complete')

Setup Complete


In [9]:
# load data

df = pd.read_csv('../data/processed_data.csv')
print(df.shape)

(6362620, 10)


In [10]:
# Train test split

X = df.drop(columns=['isFraud', 'isFlaggedFraud'])
y = df['isFraud'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [12]:
# Experiment tracking

mlflow.set_tracking_uri('https://dagshub.com/jayast29/ml-fraud-detection.mlflow')
dagshub.init(repo_owner='jayast29', repo_name='ml-fraud-detection', mlflow=True)

# mlflow.set_experiment("Random Forest")
mlflow.set_experiment("Random Forest")

Initialized MLflow to track repo "jayast29/ml-fraud-detection"

Repository jayast29/ml-fraud-detection initialized!

<Experiment: artifact_location='mlflow-artifacts:/4aded1896934487abc13f2822915ce37', creation_time=1771651302250, experiment_id='1', last_update_time=1771651302250, lifecycle_stage='active', name='Random Forest', tags={}, workspace='default'>

In [14]:
# MLflow

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)


with mlflow.start_run(run_name="random_forest_v1"):
    
    logger.info("Starting MLflow run...")
    
    # Scale
    scaler = RobustScaler()
    num_cols = X_train.select_dtypes(include=[np.number]).columns
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols])
    logger.info("Scaling complete")
    
    # Train
    model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    logger.info("Model training complete")
    
    # Evaluate
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    roc = roc_auc_score(y_test, y_prob)
    ap = average_precision_score(y_test, y_prob)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    logger.info(f"ROC-AUC: {roc:.4f} | F1: {f1:.4f}")
    
    # Log params
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("class_weight", "balanced")
    
    # Log metrics
    mlflow.log_metric("roc_auc", roc)
    mlflow.log_metric("average_precision", ap)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    mlflow.sklearn.log_model(model, "model")
    logger.info("MLflow run complete")
    
    print(classification_report(y_test, y_pred))
    print("ROC-AUC:", roc)
    print("Average Precision:", ap)

2026-02-20 22:41:23,899 - INFO - Starting MLflow run...
2026-02-20 22:41:26,224 - INFO - Scaling complete
2026-02-20 22:45:55,038 - INFO - Model training complete
2026-02-20 22:45:58,552 - INFO - ROC-AUC: 0.9624 | F1: 0.8678
2026/02/20 22:46:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/20 22:46:05 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html
2026-02-20 22:46:38,897 - WARNING - Retrying (Retry(total=6, connect=7, read=7, redirect=7, status=7)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2436)')': /jayast29/ml-fraud-detection.mlflow/api/2.0/mlflow-artifacts/art

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.98      0.78      0.87      1643

    accuracy                           1.00   1272524
   macro avg       0.99      0.89      0.93   1272524
weighted avg       1.00      1.00      1.00   1272524

ROC-AUC: 0.9624220939132065
Average Precision: 0.8679249975847507
🏃 View run random_forest_v1 at: https://dagshub.com/jayast29/ml-fraud-detection.mlflow/#/experiments/1/runs/9bd0e31aae1d4272b53389e6a844443f
🧪 View experiment at: https://dagshub.com/jayast29/ml-fraud-detection.mlflow/#/experiments/1


Random Forest performs strongly with a precision of 0.98 and recall of 0.78, meaning the model catches 78% of actual fraud while keeping false alarms extremely low. AUPRC of 0.87 and ROC-AUC of 0.96 confirm reliable fraud detection. 